In [1]:
import numpy as np

In [2]:
# useful matrices
Identity_22 = np.eye(2, dtype=np.complex128)
Pauli_x = np.array([[0, 1], [1, 0]], dtype=np.complex128)

# threshold
thr = 10**-9

In [3]:
def is_unitary(A):
    n = A.shape[0]
    if (A.shape != (n, n)):
        raise ValueError("Matrix is not square.")
    A = np.array(A)
    return np.allclose(np.eye(n), A @ A.conj().T)


def is_identity(A):
    n = A.shape[0]
    if (A.shape != (n, n)):
        raise ValueError("Matrix is not square.")
    return np.allclose(A, np.eye(n))


def elimination_matrix(a,b):
    # a, b allowed to be complex
    
    # impose theta real + positive {eq.10}
    theta = np.arctan(abs(b/a))
    
    # lambda is the negative arg() of a
    lamda = - np.angle(a)
    
    # {eq.12}
    mu = np.pi + np.angle(b)
    
    # {eq.7}
    U_special = np.array([ [np.exp(1j*lamda) * np.cos(theta), np.exp(1j*mu) * np.sin(theta)],
                           [-np.exp(-1j*mu) * np.sin(theta), np.exp(-1j*lamda) * np.cos(theta)] ])
    
    return U_special


def two_level_decomp(A):
    n = A.shape[0]
    decomp = []
    indices = []
    A_c = np.copy(A)

    for i in range(n-2):
        for j in range(n-1, i, -1):

            a = A_c[i,j-1]
            b = A_c[i,j]

            # --- need checks --- 
            # if A[i,j] = 0, nothing to do! Except in last row - need to check diagonal element is 1 
            if abs(A_c[i,j]) < thr:
                U_22 = Identity_22

                if j == i+1:
                    U_22 = np.array([[1 / a, 0], [0, a]])

            # if A[i,j-1] = 0, need to swap columns - again checking last row to ensure diagonal element is 1 
            elif abs(A_c[i,j-1]) < thr:
                U_22 = Pauli_x

                if j == i+1:
                    U_22 = np.array([[1 / b, 0], [0, b]])

            # Special unitary matrix
            else: 
                U_22 = elimination_matrix(a,b)

            # ----- U_22 found -----

            # multiply submatrix of A with U_22
            A_c[:,(j-1,j)] = A_c[:,(j-1,j)] @ U_22

            # If not the identity matrix - represents a gate! So should store
            if not is_identity(U_22):
                decomp.append(U_22.conj().T)
                indices.append(np.array([j-1,j]))


        # check for diagonal element equal to 1
        assert np.allclose(A_c[i,i],1.0)
    
    # lower right hand 2x2 matrix remaining after decomp
    lower_rh_matrix = A_c[n-2:n, n-2:n]
    
    # if not equal to I - is a non trivial gate
    if not is_identity(lower_rh_matrix):
        decomp.append(lower_rh_matrix)
        indices.append(np.array([n-2,n-1]))

    return decomp, indices


def gray_method(A):
    
    n = A.shape[0]
#     A = np.array(A)
    # using bitwise_xor find Gray permutations
    permutations = []
    for i in range(n):
        permutations.append(i ^ (i // 2))
        
    # 
    A[:,:] = A[:,permutations]
    A[:,:] = A[permutations,:]
    
    decomp, indices = two_level_decomp(A)
    new_ind = []
    
    for pair in indices:
        new_ind.append(np.take(permutations, pair, 0))
        
        
    return decomp, new_ind
    

In [4]:
# test matrix
w = np.exp((2j / 3) * np.pi)


In [5]:
A = np.array([[1, 1, 1, 0], 
                  [1, w, w * w, 0],
                  [1, w * w, w, 0], 
                  [0, 0, 0, -1j*np.sqrt(3)]]) / np.sqrt(3)
two_level_decomp(A)

([array([[ 0.70710678-0.00000000e+00j,  0.70710678-8.65956056e-17j],
         [-0.70710678-8.65956056e-17j,  0.70710678-0.00000000e+00j]]),
  array([[ 0.57735027-0.00000000e+00j,  0.81649658-9.99919924e-17j],
         [-0.81649658-9.99919924e-17j,  0.57735027-0.00000000e+00j]]),
  array([[-7.07106781e-01+8.65956056e-17j, -5.14325540e-16-7.07106781e-01j],
         [ 5.14325540e-16-7.07106781e-01j, -7.07106781e-01-8.65956056e-17j]]),
  array([[-6.98727119e-16-1.j,  0.00000000e+00+0.j],
         [ 0.00000000e+00+0.j,  0.00000000e+00-1.j]])],
 [array([1, 2]), array([0, 1]), array([1, 2]), array([2, 3])])

In [7]:
A = np.array([[1, 1, 1, 0], 
                  [1, w, w * w, 0],
                  [1, w * w, w, 0], 
                  [0, 0, 0, -1j*np.sqrt(3)]]) / np.sqrt(3)
gray_method(A)

([array([[0.-0.j, 1.-0.j],
         [1.-0.j, 0.-0.j]]),
  array([[ 0.70710678-0.00000000e+00j,  0.70710678-8.65956056e-17j],
         [-0.70710678-8.65956056e-17j,  0.70710678-0.00000000e+00j]]),
  array([[ 0.57735027-0.00000000e+00j,  0.81649658-9.99919924e-17j],
         [-0.81649658-9.99919924e-17j,  0.57735027-0.00000000e+00j]]),
  array([[-7.07106781e-01+8.65956056e-17j, -5.14325540e-16-7.07106781e-01j],
         [ 5.14325540e-16-7.07106781e-01j, -7.07106781e-01-8.65956056e-17j]]),
  array([[ 0.00000000e+00+0.j,  0.00000000e+00-1.j],
         [-6.98727119e-16-1.j,  0.00000000e+00+0.j]])],
 [array([3, 2]), array([1, 3]), array([0, 1]), array([1, 3]), array([3, 2])])

In [8]:
from scipy.stats import unitary_group
nq = 2
A = unitary_group.rvs(2**nq)
decomp, indices = two_level_decomp(A)
decomp

[array([[-0.54303015-0.73182587j, -0.38818761-0.13733002j],
        [ 0.38818761-0.13733002j, -0.54303015+0.73182587j]]),
 array([[-0.13532716+3.93774545e-01j,  0.90919094-1.11343778e-16j],
        [-0.90919094-1.11343778e-16j, -0.13532716-3.93774545e-01j]]),
 array([[-0.09915216+5.70090997e-01j,  0.81557655-9.98793207e-17j],
        [-0.81557655-9.98793207e-17j, -0.09915216-5.70090997e-01j]]),
 array([[-0.86965324-0.46145279j,  0.10483577+0.14062014j],
        [-0.10483577+0.14062014j, -0.86965324+0.46145279j]]),
 array([[-0.05321439+1.29527182e-01j,  0.99014693-1.21258027e-16j],
        [-0.99014693-1.21258027e-16j, -0.05321439-1.29527182e-01j]]),
 array([[-0.08860447+0.20611519j,  0.96346667-0.1462797j ],
        [ 0.17275923-0.95907248j,  0.20847727-0.0828942j ]])]